In [2]:
import xarray as xr
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import numpy as np
import xesmf as xe
from pathlib import Path

Cargamos los modelos en un dataxarray

In [3]:
def abrir_modelos(variable, ruta_base=None):
    """
    Abrir todos los archivos .nc en la subcarpeta correspondiente a la variable dentro de 'modelos'.

    Returns:
      list[xarray.Dataset]: lista con los datasets abiertos.

    Raises:
      FileNotFoundError: si la carpeta no existe o no contiene archivos .nc.
    """
    try:
        base = Path(__file__).parent  # disponible si se ejecuta como script
    except NameError:
        base = Path.cwd()
    # Resolver ruta_base: por defecto 'modelos' relativa a este archivo,
    # pero permite pasar una ruta absoluta o relativa si se desea.
    if ruta_base:
        modelos_root = Path(ruta_base)
        if not modelos_root.is_absolute():
            modelos_root = base / modelos_root
    else:
        modelos_root = base / 'modelos'

    ruta_variable = modelos_root / variable
    if not ruta_variable.exists() or not ruta_variable.is_dir():
        raise FileNotFoundError(f"No existe la carpeta de la variable: {ruta_variable}")

    nc_files = sorted(ruta_variable.glob('*.nc'))
    if not nc_files:
        raise FileNotFoundError(f"No se encontraron archivos .nc en {ruta_variable}")

    modelos = [xr.open_dataset(str(archivo)) for archivo in nc_files]

    print(f"Se abrieron {len(modelos)} modelos para la variable '{variable}'.")
    return modelos

In [16]:
modelos_clt = abrir_modelos('clt')
modelos_pr = abrir_modelos('pr')
modelos_rsds = abrir_modelos('rsds')
modelos_sfcWind = abrir_modelos('sfcWind')
modelos_tas = abrir_modelos('tas')

c:\Users\123\anaconda3\envs\clima\Lib\site-packages\xarray\conventions.py:204: SerializationWarning: variable 'clt' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Se abrieron 7 modelos para la variable 'clt'.


c:\Users\123\anaconda3\envs\clima\Lib\site-packages\xarray\conventions.py:204: SerializationWarning: variable 'pr' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Se abrieron 7 modelos para la variable 'pr'.


c:\Users\123\anaconda3\envs\clima\Lib\site-packages\xarray\conventions.py:204: SerializationWarning: variable 'rsds' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Se abrieron 7 modelos para la variable 'rsds'.


c:\Users\123\anaconda3\envs\clima\Lib\site-packages\xarray\conventions.py:204: SerializationWarning: variable 'sfcWind' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Se abrieron 7 modelos para la variable 'sfcWind'.


c:\Users\123\anaconda3\envs\clima\Lib\site-packages\xarray\conventions.py:204: SerializationWarning: variable 'tas' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Se abrieron 7 modelos para la variable 'tas'.


Como los modelos tienen diferentes tamamaños de grids, vemos el tamaño de los grids de cada modelo y lo convertimos todos en el mismo tamaño

In [19]:
def info_grids(modelos):
    """
    Dada una lista de modelos (xarray.Dataset),
    devuelve un resumen con el tamaño y nombre de las coordenadas de cada grid.
    """
    info = []
    for i, ds in enumerate(modelos, start=1):
        # Buscar posibles nombres de coordenadas de lat/lon
        lat_name = next((c for c in ds.coords if 'lat' in c.lower()), None)
        lon_name = next((c for c in ds.coords if 'lon' in c.lower()), None)
        
        if lat_name and lon_name:
            lat_size = ds[lat_name].size
            lon_size = ds[lon_name].size
            info.append({
                'modelo': i,
                'lat_name': lat_name,
                'lon_name': lon_name,
                'lat_size': lat_size,
                'lon_size': lon_size
            })
        else:
            info.append({
                'modelo': i,
                'lat_name': lat_name,
                'lon_name': lon_name,
                'error': 'No se encontraron coordenadas lat/lon'
            })
    
    return info


Con mirar la lista para uno es suficiente ya que al usar los mismos modelos, en todos tendrían que tener el mismo tamaño

In [20]:
grids_info = info_grids(modelos_clt)
for g in grids_info:
    print(g)

{'modelo': 1, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 144, 'lon_size': 192}
{'modelo': 2, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 145, 'lon_size': 192}
{'modelo': 3, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 4, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 5, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 6, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 160, 'lon_size': 320}
{'modelo': 7, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}


In [ ]:
def regrid_all(modelos, var_name, ref_index):
    """
    Regridea todas las variables 'var_name' de los modelos (en una lista)
    al grid del modelo de referencia indicado por ref_index.

    Parámetros
    ----------
    modelos : list[xarray.Dataset | xarray.DataArray]
        Lista de modelos.
    var_name : str
        Nombre de la variable a regridear (ej: 'clt')
    ref_index : int
        Índice del modelo de referencia en la lista (ej: 2 para el tercero)
    
    Retorna
    -------
    list[xarray.DataArray]
        Lista de variables regrideadas al mismo grid.
    """
    regridded = []
    ref_model = modelos[ref_index]
    ref_grid = ref_model[var_name] if hasattr(ref_model, 'data_vars') else ref_model

    for i, model in enumerate(modelos):
        print(f"Procesando modelo {i} de {var_name}...")
        data = model[var_name] if hasattr(model, 'data_vars') else model
        
        if i == ref_index:
            regridded.append(data)
        else:
            regridder = xe.Regridder(
                data,
                ref_grid,
                method='bilinear',
                extrap_method='nearest_s2d',
                reuse_weights=False
            )
            regridded.append(regridder(data))
    
    return regridded

In [22]:
clt_regridded = regrid_all(modelos_clt, 'clt', 3)
pr_regridded = regrid_all(modelos_pr, 'pr', 3)
rsds_regridded = regrid_all(modelos_rsds, 'rsds', 3)
sfcWind_regridded = regrid_all(modelos_sfcWind, 'sfcWind', 3)
tas_regridded = regrid_all(modelos_tas, 'tas', 3)


Procesando modelo 0...
Procesando modelo 1...
Procesando modelo 2...
Procesando modelo 3...
Procesando modelo 4...
Procesando modelo 5...
Procesando modelo 6...
Procesando modelo 0...
Procesando modelo 1...
Procesando modelo 2...
Procesando modelo 3...
Procesando modelo 4...
Procesando modelo 5...
Procesando modelo 6...
Procesando modelo 0...
Procesando modelo 1...
Procesando modelo 2...
Procesando modelo 3...
Procesando modelo 4...
Procesando modelo 5...
Procesando modelo 6...
Procesando modelo 0...
Procesando modelo 1...
Procesando modelo 2...
Procesando modelo 3...
Procesando modelo 4...
Procesando modelo 5...
Procesando modelo 6...
Procesando modelo 0...
Procesando modelo 1...
Procesando modelo 2...
Procesando modelo 3...
Procesando modelo 4...
Procesando modelo 5...
Procesando modelo 6...


Comprobar que efectivamente lo hicimos bien y tienen todos tamaño: 192x288

In [23]:
primeros = [
    clt_regridded[0],
    pr_regridded[0],
    rsds_regridded[0],
    sfcWind_regridded[0],
    tas_regridded[0]
]
grids_info = info_grids(primeros)
for g in grids_info:
    print(g)


{'modelo': 1, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 2, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 3, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 4, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 5, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}


Calculo de medias mensuales

In [27]:
def calc_monthly_means(modelos, var_name=None, start='1950-01-01', end='1980-12-31'):
    """
    Calcula la media mensual (1950–1980 por defecto) para una lista de modelos.
    
    Parámetros
    ----------
    modelos : list[xarray.Dataset | xarray.DataArray]
        Lista de modelos o variables.
    var_name : str | None
        Nombre de la variable si los elementos son Datasets (ej: 'clt').
        Si ya son DataArray, déjalo en None.
    start, end : str
        Fechas para el rango temporal (slice).
    
    Retorna
    -------
    list[xarray.DataArray]
        Lista con las medias mensuales de cada modelo.
    """
    monthly_means = []
    
    for i, model in enumerate(modelos):

        # Extrae la variable si es Dataset
        data = model[var_name] if (var_name and hasattr(model, 'data_vars')) else model
        
        # Selección temporal y promedio mensual
        mean_month = (
            data
            .sel(time=slice(start, end))
            .groupby('time.month')
            .mean('time')
        )
        
        monthly_means.append(mean_month)
    
    return monthly_means


In [28]:
clt_monthly_means = calc_monthly_means(clt_regridded, start='1950-01-01', end='1980-12-31')
pr_monthly_means = calc_monthly_means(pr_regridded, start='1950-01-01', end='1980-12-31')
rsds_monthly_means = calc_monthly_means(rsds_regridded, start='1950-01-01', end='1980-12-31')
sfcWind_monthly_means = calc_monthly_means(sfcWind_regridded, start='1950-01-01', end='1980-12-31')
tas_monthly_means = calc_monthly_means(tas_regridded, start='1950-01-01', end='1980-12-31')


Ahora calculamos la medía del ensemble, es decir, entre todos los modelos que tenemos para cada variable sacamos la medía

In [30]:
def ensemble_mean(models_monthly):
    """
    Calcula la media del ensemble (promedio entre modelos).
    
    Parámetros
    ----------
    models_monthly : list[xarray.DataArray]
        Lista de DataArrays con las medias mensuales de cada modelo.
    
    Retorna
    -------
    xarray.DataArray
        DataArray con el promedio del ensemble.
    """
    # Concatenamos a lo largo de un eje ficticio 'model'
    combined = xr.concat(models_monthly, dim='model')
    return combined.mean(dim='model')

In [33]:
clt_ensemble_mean = ensemble_mean(clt_monthly_means)
pr_ensemble_mean = ensemble_mean(pr_monthly_means)
rsds_ensemble_mean = ensemble_mean(rsds_monthly_means)
sfcWind_ensemble_mean = ensemble_mean(sfcWind_monthly_means)
tas_ensemble_mean = ensemble_mean(tas_monthly_means)

Convertimos los resultados a un dataframe para poder importarlos a un .csv y .cn

In [40]:

# Creamos las combinaciones lat-lon-time a partir de uno de los DataArray
lat = clt_ensemble_mean['lat'].values
lon = clt_ensemble_mean['lon'].values
time = clt_ensemble_mean['month'].values

# Crear el índice combinando todas las coordenadas
import itertools
index = [f"{t}_{la}_{lo}" for t, la, lo in itertools.product(time, lat, lon)]

In [41]:
df = pd.DataFrame({
    'clt_ensemble_mean': clt_ensemble_mean.values.flatten(),
    'pr_ensemble_mean': pr_ensemble_mean.values.flatten(),
    'rsds_ensemble_mean': rsds_ensemble_mean.values.flatten(),
    'sfcWind_ensemble_mean': sfcWind_ensemble_mean.values.flatten(),
    'tas_ensemble_mean': tas_ensemble_mean.values.flatten()
}, index=index)

In [42]:
df.to_csv('ensemble_modelos.csv', index=True)